In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
%run utils/mlflow_query.py
%run utils/loading.py
%run utils/comparison.py
%run utils/percentiles.py

In [ ]:
mlflow_helper = MlflowHelper(pkl_file=Path("mlflow_run_df.pkl"))
#mlflow_helper.query_all_runs(pkl_file=Path("mlflow_run_df.pkl"))

In [ ]:
percentile_names=[
    'avg_input_frequencies_percentile', 
    'median_input_frequencies_percentile', 
    'min_input_frequencies_percentile', 
    'p10_input_frequencies_percentile', 
    'unknown_inputs_percentile', 
    'output_frequency_percentile',
    'avg_input_frequencies_range', 
    'median_input_frequencies_range', 
    'min_input_frequencies_range', 
    'p10_input_frequencies_range', 
    'unknown_inputs_range', 
    "unclustered_inputs_percentile",
    "unclustered_inputs_perc_percentile",
    "clustered_inputs_percentile",
    "clustered_inputs_perc_percentile",
    "single_clustered_inputs_percentile",
    "single_clustered_inputs_perc_percentile",
    "single_clustered_inputs_clusterperc_percentile",
    "multi_clustered_inputs_percentile",
    "multi_clustered_inputs_perc_percentile",
    "multi_clustered_inputs_clusterperc_percentile",
    "avg_cluster_input_frequency_percentile",
    "median_cluster_input_frequency_percentile",
    "unclustered_inputs_range",
    "unclustered_inputs_perc_range",
    "clustered_inputs_range",
    "clustered_inputs_perc_range",
    "single_clustered_inputs_range",
    "single_clustered_inputs_perc_range",
    "single_clustered_inputs_clusterperc_range",
    "multi_clustered_inputs_range",
    "multi_clustered_inputs_perc_range",
    "multi_clustered_inputs_clusterperc_range",
    "avg_cluster_input_frequency_range",
    "median_cluster_input_frequency_range",
]

# Overall accuracy per percentiles

In [ ]:
relevant_mimic_run_df = mlflow_helper.mimic_run_df(include_noise=False, include_refinements=False)
mimic_accuracy_df = calculate_accuracies_per_percentiles(
    relevant_run_df=relevant_mimic_run_df, 
    k=20, num_percentiles=10, num_input_percentiles=10,
    percentile_names=percentile_names,
    cluster_threshold=0.5,
    local_mlflow_dir=mlflow_helper.local_mlflow_dir)

In [ ]:
merged_original = pd.merge(
    relevant_mimic_run_df, 
    mimic_accuracy_df, 
    left_on="info_run_id", 
    right_on="run_id")
simple_false_run_ids = set(merged_original[
    (merged_original["data_tags_model_type"] == "simple")
    & (merged_original["data_params_ModelConfigbase_feature_embeddings_trainable"] == "False")
]["info_run_id"])
model_false_run_ids = set(merged_original[
    (merged_original["data_tags_model_type"].apply(lambda x: x in ["gram", "text", "causal", "causal2"]))
    & (merged_original["data_params_ModelConfigbase_feature_embeddings_trainable"] == "False")
]["info_run_id"])

types = [
    "avg_input_frequencies_percentile", 
    "median_input_frequencies_percentile", 
    "unknown_inputs_percentile",
    "median_cluster_input_frequency_percentile",
    "avg_cluster_input_frequency_percentile"]
percentiles = range(10)
improvements = []
for model_false_run_id in tqdm(model_false_run_ids):
    for simple_false_run_id in simple_false_run_ids:
        for type in types:
            for percentile in percentiles:
                accuracy_imp = merged_original[
                    (merged_original["info_run_id"] == model_false_run_id)
                    & (merged_original["type"] == type)
                    & (merged_original["percentile"] == percentile)
                ]["accuracy"].iloc[0] - merged_original[
                    (merged_original["info_run_id"] == simple_false_run_id)
                    & (merged_original["type"] == type)
                    & (merged_original["percentile"] == percentile)
                ]["accuracy"].iloc[0]
                improvements.append({
                    "model_run_id": model_false_run_id,
                    "simple_run_id": simple_false_run_id,
                    "accuracy_imp": accuracy_imp,
                    "type": type,
                    "percentile": percentile,
                })

In [ ]:
merged = pd.merge(
    relevant_mimic_run_df, 
    pd.DataFrame.from_records(improvements), 
    left_on="info_run_id", 
    right_on="model_run_id")
merged["Domain Knowledge"] = merged["data_tags_model_type"].apply(lambda x: {
    "gram":"hierarchy",
    "causal2": "causal",
}.get(x,x))
g = sns.relplot(
    data=merged[
        (merged["percentile"] > -1) 
        & (merged["data_params_ModelConfigbase_feature_embeddings_trainable"] == "False")
        & (merged["type"] == "median_cluster_input_frequency_percentile")
    ],
    x="percentile",
    y="accuracy_imp",
    #row="type",
    hue="Domain Knowledge",
    #col="data_params_ModelConfigbase_hidden_embeddings_trainable",
    kind="line",
    palette="Set2",
    hue_order=["hierarchy", "causal", "text"]
    #facet_kws={"sharey": "row", "sharex": True},
    #col_order=["False", "True"],
).map(plt.axhline, y=0, color=".7", dashes=(2, 1), zorder=0).set_titles("").set_axis_labels(
    "median_cluster_input_frequency_percentile", "val_top_20_categorical_accuracy_diff"
)#.set_xlabels("avg_input_frequency_percentile").set_ylabels("val_top_20_categorical_accuracy_diff")
for ax in g.axes.flatten():
    ax.tick_params(labelbottom=True)
#plt.savefig("percentiles_healthcare_avg.png", dpi=100, bbox_inches="tight")
plt.show()

In [ ]:
merged = pd.merge(
    relevant_mimic_run_df, 
    pd.DataFrame.from_records(improvements), 
    left_on="info_run_id", 
    right_on="model_run_id")
merged["Domain Knowledge"] = merged["data_tags_model_type"].apply(lambda x: {
    "gram":"hierarchy",
    "causal2": "causal",
}.get(x,x))
g = sns.relplot(
    data=merged[
        (merged["percentile"] > -1) 
        & (merged["data_params_ModelConfigbase_feature_embeddings_trainable"] == "False")
        & (merged["type"] == "avg_input_frequencies_percentile")
    ],
    x="percentile",
    y="accuracy_imp",
    #row="type",
    hue="Domain Knowledge",
    #col="data_params_ModelConfigbase_hidden_embeddings_trainable",
    kind="line",
    palette="Set2",
    hue_order=["hierarchy", "causal", "text"]
    #facet_kws={"sharey": "row", "sharex": True},
    #col_order=["False", "True"],
).map(plt.axhline, y=0, color=".7", dashes=(2, 1), zorder=0).set_titles("").set_axis_labels(
    "avg_input_frequencies_percentile", "val_top_20_categorical_accuracy_diff"
)#.set_xlabels("avg_input_frequency_percentile").set_ylabels("val_top_20_categorical_accuracy_diff")
for ax in g.axes.flatten():
    ax.tick_params(labelbottom=True)
plt.savefig("percentiles_healthcare_avg.png", dpi=100, bbox_inches="tight")

plt.show()

In [ ]:
relevant_run_df = relevant_mimic_run_df
accuracy_df = mimic_accuracy_df2.copy()
merged = pd.merge(relevant_run_df, accuracy_df, left_on="info_run_id", right_on="run_id_comp")
merged["accuracy_imp"] = merged["accuracy_comp"] - merged["accuracy_base"]

g = sns.relplot(
    data=merged[
        (merged["percentile"] > -1) 
    ],
    x="percentile",
    y="accuracy_imp",
    row="type",
    hue="data_tags_model_type",
    col="data_params_ModelConfigbase_hidden_embeddings_trainable",
    kind="line",
    palette=None,
    facet_kws={"sharey": "row", "sharex": True},
    col_order=["False", "True"],
).set_titles("Type: {row_name}, Comparison: {col_name}").set_axis_labels(
    "", "mean_accuracy_diff"
)
g.map(plt.axhline, y=0, color=".7", dashes=(2, 1), zorder=0)
for ax in g.axes.flatten():
    ax.tick_params(labelbottom=True)
plt.show()

In [ ]:
plot_accuracies_per_percentiles(
    relevant_run_df=relevant_mimic_run_df, 
    accuracy_df=mimic_accuracy_df,
        #mimic_accuracy_df["type"].apply(lambda x: x in [
        #    "avg_cluster_input_frequency_percentile",
        #    "median_cluster_input_frequency_percentile",
        #])
    #],
    show_plot=True,
    share_y=True
)

In [ ]:
relevant_huawei_run_df = mlflow_helper.huawei_run_df(include_noise=False, include_refinements=False)
huawei_value_df = calculate_values_per_percentiles(
    relevant_run_df=relevant_huawei_run_df, 
    num_percentiles=10, 
    percentile_names=[
        (percentile_name, percentile_name.replace("_range", "").replace("_percentile", ""))
        for percentile_name in percentile_names
        if not "output_frequency" in percentile_name
        and percentile_name.endswith("_percentile")
    ],
    cluster_threshold=0.5,
    local_mlflow_dir=mlflow_helper.local_mlflow_dir)

In [ ]:
relevant_mimic_run_df = mlflow_helper.mimic_run_df(include_noise=False, include_refinements=False)
mimic_value_df = calculate_values_per_percentiles(
    relevant_run_df=relevant_mimic_run_df, 
    num_percentiles=10, 
    percentile_names=[
        (percentile_name, percentile_name.replace("_range", "").replace("_percentile", ""))
        for percentile_name in percentile_names
        if not "output_frequency" in percentile_name
        and percentile_name.endswith("_percentile")
    ],
    cluster_threshold=0.5,
    local_mlflow_dir=mlflow_helper.local_mlflow_dir)

In [ ]:
mimic_accuracy_value_df = pd.merge(mimic_accuracy_df, mimic_value_df, on=["percentile", "run_id"], suffixes=("_accuracy", "_value"))
mimic_accuracy_value_df = pd.merge(mimic_accuracy_value_df, relevant_mimic_run_df, left_on="run_id", right_on="info_run_id")
mimic_accuracy_value_df = mimic_accuracy_value_df[
    mimic_accuracy_value_df["type_accuracy"] == mimic_accuracy_value_df["type_value"].apply(lambda x: x+"_percentile")
]
mimic_accuracy_value_df

In [ ]:
sns.relplot(
    data=mimic_accuracy_value_df[
        mimic_accuracy_value_df['percentile'].apply(lambda x: int(x) > -1)
        & mimic_accuracy_value_df['data_params_ModelConfigbase_feature_embeddings_trainable'].apply(lambda x: x == "False")
    ],
    x='percentile',
    y='accuracy',
    col='data_tags_model_type',
    col_order=["simple", "gram", "causal", "causal2", "text"],
    row='type_accuracy',
    kind='line',
    #facet_kws={'sharey': False, 'sharex': True},
)
plt.show()

In [ ]:
sns.relplot(
    data=mimic_accuracy_value_df[
        mimic_accuracy_value_df['percentile'].apply(lambda x: int(x) > -1)
        & mimic_accuracy_value_df['data_params_ModelConfigbase_feature_embeddings_trainable'].apply(lambda x: x == "False")
    ],
    x='percentile',
    y='value',
    col='data_tags_model_type',
    col_order=["simple", "gram", "causal", "causal2", "text"],
    row='type_accuracy',
    kind='line',
    facet_kws={'sharey': "row", 'sharex': True},
)
plt.show()

In [ ]:
m2 = pd.merge(
    relevant_mimic_run_df,
    mimic_value_df,
    left_on="info_run_id",
    right_on="run_id",
)
sns.relplot(
    data=m2[
        (m2["percentile"] > -1) 
        & (m2["data_params_ModelConfigbase_feature_embeddings_trainable"] == "False")
        & (m2["data_params_SequenceConfigx_sequence_column_name"] == "level_0")
        & (m2["type"] == "median_cluster_input_frequency")
    ],
    x="percentile",
    y="value",
    row="type",
    hue="data_tags_model_type",
    #col="data_params_ModelConfigbase_hidden_embeddings_trainable",
    kind="line",
    palette="Set2",
    facet_kws={"sharey": "row", "sharex": True},
)
plt.show()

In [ ]:
grouped_df = (
    pd.merge(relevant_mimic_run_df, mimic_value_df, left_on="info_run_id", right_on="run_id")
    .groupby(
        [
            "data_tags_model_type",
            "data_params_ModelConfigbase_hidden_embeddings_trainable",
            "info_run_id",
            "type",
            "percentile",
        ],
        as_index=False,
    )
    .agg({"value": max,})
)
sns.relplot(
    data=grouped_df,
    x="percentile",
    y="value",
    row="type",
    hue="data_tags_model_type",
    col="data_params_ModelConfigbase_hidden_embeddings_trainable",
    kind="line",
    palette=None,
    facet_kws={"sharey": "row", "sharex": True},
)
plt.show()

In [ ]:
plot_accuracies_per_percentiles(
    relevant_run_df=relevant_mimic_run_df, 
    accuracy_df=mimic_accuracy_df[
        mimic_accuracy_df["type"].apply(lambda x: x in [
            "clustered_inputs_perc_percentile",
            "single_clustered_inputs_perc_percentile",
            "multi_clustered_inputs_perc_percentile"
        ])
    ],
    show_plot=True,
    share_y=True
)

In [ ]:
rel_df_huawei_1 = mlflow_helper.huawei_run_df(include_noise=False, include_refinements=False, valid_x_columns=["log_cluster_template", "fine_log_cluster_template", "coarse_log_cluster_template"])
#accuracy_df_1 = calculate_accuracies_per_percentiles(
#    relevant_run_df=rel_df_1, k=5, num_percentiles=10, num_input_percentiles=10,
#    percentile_names=percentile_names,
#    local_mlflow_dir=mlflow_helper.local_mlflow_dir)
#accuracy_df_1.to_csv("accuracy_df_1.csv")
accuracy_df_huawei_1 = pd.read_csv("accuracy_df_1.csv")

In [ ]:
rel_df_mimic_2 = mlflow_helper.mimic_run_df(include_noise=False, include_refinements=True, valid_x_columns=["level_0", "level_1", "level_2"])
#accuracy_df_2 = calculate_accuracies_per_percentiles(
#    relevant_run_df=rel_df_2, k=20, num_percentiles=10, num_input_percentiles=10,
#    percentile_names=percentile_names,
#    local_mlflow_dir=mlflow_helper.local_mlflow_dir)
#accuracy_df_2.to_csv("accuracy_df_2.csv")
accuracy_df_mimic_2 = pd.read_csv("accuracy_df_2.csv")

In [ ]:
m2 = pd.merge(
    rel_df_mimic_2,
    accuracy_df_mimic_2,
    left_on="info_run_id",
    right_on="run_id",
)
sns.relplot(
    data=m2[
        (m2["percentile"] > -1) 
        & (m2["data_params_ModelConfigbase_feature_embeddings_trainable"] == "False")
        & (m2["data_params_SequenceConfigx_sequence_column_name"] == "level_0")
        & (m2["type"] == "avg_cluster_input_frequency_percentile")
    ],
    x="percentile",
    y="accuracy",
    row="type",
    hue="data_tags_model_type",
    #col="data_params_ModelConfigbase_hidden_embeddings_trainable",
    kind="line",
    palette="Set2",
    facet_kws={"sharey": "row", "sharex": True},
)
plt.show()

In [ ]:
relevant_huawei_run_df = mlflow_helper.huawei_run_df(include_noise=False, include_refinements=False)
huawei_accuracy_df = calculate_accuracies_per_percentiles(
    relevant_run_df=relevant_huawei_run_df, k=5, num_percentiles=10, num_input_percentiles=10,
    percentile_names=percentile_names,
    local_mlflow_dir=mlflow_helper.local_mlflow_dir)

In [ ]:
merged_original = pd.merge(
    mlflow_helper.mimic_run_df(include_noise=False, include_refinements=False, valid_x_columns=["level_0", "level_1", "level_2"]), 
    accuracy_df_mimic_2, 
    left_on="info_run_id", 
    right_on="run_id")
simple_false_run_ids = set(merged_original[
    (merged_original["data_tags_model_type"] == "simple")
    & (merged_original["data_params_ModelConfigbase_feature_embeddings_trainable"] == "False")
]["info_run_id"])
model_false_run_ids = set(merged_original[
    (merged_original["data_tags_model_type"].apply(lambda x: x in ["gram", "text", "causal", "causal2"]))
    & (merged_original["data_params_ModelConfigbase_feature_embeddings_trainable"] == "False")
]["info_run_id"])

types = ["median_input_frequencies_percentile", "avg_input_frequencies_percentile", "unknown_inputs_percentile"]
percentiles = range(10)
improvements = []
for model_false_run_id in tqdm(model_false_run_ids):
    for simple_false_run_id in simple_false_run_ids:
        for type in types:
            for percentile in percentiles:
                accuracy_imp = merged_original[
                    (merged_original["info_run_id"] == model_false_run_id)
                    & (merged_original["type"] == type)
                    & (merged_original["percentile"] == percentile)
                ]["accuracy"].iloc[0] - merged_original[
                    (merged_original["info_run_id"] == simple_false_run_id)
                    & (merged_original["type"] == type)
                    & (merged_original["percentile"] == percentile)
                ]["accuracy"].iloc[0]
                improvements.append({
                    "model_run_id": model_false_run_id,
                    "simple_run_id": simple_false_run_id,
                    "accuracy_imp": accuracy_imp,
                    "type": type,
                    "percentile": percentile,
                })

In [ ]:
merged = pd.merge(
    mlflow_helper.mimic_run_df(include_noise=False, include_refinements=False, valid_x_columns=["level_0", "level_1", "level_2"]), 
    pd.DataFrame.from_records(improvements), 
    left_on="info_run_id", 
    right_on="model_run_id")
merged = pd.merge(
    mlflow_helper.mimic_run_df(include_noise=False, include_refinements=False, valid_x_columns=["level_0", "level_1", "level_2"]), 
    merged, 
    left_on="info_run_id", 
    right_on="simple_run_id",
    suffixes=("_simple",""))
merged["Expert Knowledge"] = merged["data_tags_model_type"].apply(lambda x: {
    "gram":"hierarchy",
    #"causal2": "causal",
    #"causal": "causal_old",
}.get(x,x))
g = sns.relplot(
    data=merged[
        (merged["percentile"] > -1) 
        & (merged["data_params_ModelConfigbase_feature_embeddings_trainable_simple"] == "False")
        & (merged["data_params_ModelConfigbase_feature_embeddings_trainable"] == "False")
        & (merged["data_tags_model_type_simple"] == "simple")
        & (merged["data_params_SequenceConfigx_sequence_column_name_simple"] == "level_0")#merged["data_params_SequenceConfigx_sequence_column_name"])
        & (merged["type"] == "avg_input_frequencies_percentile")
    ],
    x="percentile",
    y="accuracy_imp",
    #row="type",
    hue="Expert Knowledge",
    col="data_params_SequenceConfigx_sequence_column_name",
    kind="line",
    palette="Set2",
    hue_order=["hierarchy", "causal", "text"]
    #facet_kws={"sharey": "row", "sharex": True},
    #col_order=["False", "True"],
).map(plt.axhline, y=0, color=".7", dashes=(2, 1), zorder=0).set_titles("Input granularity: {col_name}").set_axis_labels(
    "avg_input_frequencies_percentile", "val_top_20_categorical_accuracy_diff"
).set_xlabels("avg_input_frequency_percentile").set_ylabels("val_top_20_categorical_accuracy_diff")
for ax in g.axes.flatten():
    ax.tick_params(labelbottom=True)
plt.savefig("percentiles_mimic_avg_granularities.png", dpi=100, bbox_inches="tight")
plt.show()

In [ ]:
plot_accuracies_per_percentiles(
    relevant_run_df=relevant_huawei_run_df, 
    accuracy_df=huawei_accuracy_df[
        huawei_accuracy_df["type"].apply(lambda x: x in [
            "clustered_inputs_perc_percentile",
            "single_clustered_inputs_perc_percentile",
            "multi_clustered_inputs_perc_percentile"
        ])    
    ],
    show_plot=True,
    share_y=True
)

# Accuracy@ for different input granularities

In [ ]:
relevant_mimic_run_df2 = mlflow_helper.mimic_run_df(include_noise=False, include_refinements=False, 
    valid_x_columns=["level_0", "level_1", "level_2"])
mimic_accuracy_df2 = calculate_accuracies_per_percentiles(
    relevant_run_df=relevant_mimic_run_df2, 
    k=20, num_percentiles=10, num_input_percentiles=10,
    percentile_names=percentile_names,
    local_mlflow_dir=mlflow_helper.local_mlflow_dir)

In [ ]:
merged_df = pd.merge(
    relevant_mimic_run_df2[
        relevant_mimic_run_df2["data_params_ModelConfigbase_hidden_embeddings_trainable"] == "False"
    ], mimic_accuracy_df2[
        mimic_accuracy_df2["type"] == "output_frequency_percentile"
    ], left_on="info_run_id", right_on="run_id")
sns.catplot(
    data=merged_df,
    x="data_tags_model_type",
    order=["simple", "gram", "text", "causal"],
    y="accuracy",
    kind="box",
    row="percentile",
    col="data_params_SequenceConfigx_sequence_column_name",
    col_order=["level_0", "level_1", "level_2"],
    sharey="row",
).set_titles("Percentile: {row_name}, X = {col_name}").set_axis_labels(
    "", "accuracy"
)
plt.show()

In [ ]:
plot_accuracies_per_percentiles(
    relevant_run_df=relevant_mimic_run_df2[
        relevant_mimic_run_df2["data_params_ModelConfigbase_hidden_embeddings_trainable"] == "False"
    ], 
    accuracy_df=mimic_accuracy_df2[
        (mimic_accuracy_df2["type"] == "output_frequency_percentile")
        & (mimic_accuracy_df2["percentile"].astype(int) > -1)
    ],
    comparison_column="data_params_SequenceConfigx_sequence_column_name",
    comparison_column_order=["level_0", "level_1", "level_2"],
    show_plot=True,
    share_y=True)

In [ ]:
grouped_df = (
    pd.merge(
        relevant_mimic_run_df2[relevant_mimic_run_df2['data_tags_model_type'] != 'causal2'], 
        mimic_accuracy_df2[mimic_accuracy_df2['type'] == 'avg_input_frequencies_percentile'], left_on="info_run_id", right_on="run_id")
    .groupby(
        [
            "data_tags_model_type",
            "data_params_ModelConfigbase_hidden_embeddings_trainable",
            "data_params_SequenceConfigx_sequence_column_name",
            "info_run_id",
            "type",
            "percentile",
        ],
        as_index=False,
    )
    .agg({"accuracy": max,})
)
g = sns.relplot(
    data=grouped_df,
    x="percentile",
    y="accuracy",
    row="type",
    col="data_tags_model_type",
    hue="data_params_SequenceConfigx_sequence_column_name",
    style="data_params_ModelConfigbase_hidden_embeddings_trainable",
    kind="line",
    palette=None,
)
g.set_titles("Type: {row_name}, Model: {col_name}").set_axis_labels(
    "", "accuracy"
)
for ax in g.axes.flatten():
    ax.tick_params(labelbottom=True)
plt.show()

In [ ]:
relevant_huawei_run_df2 = mlflow_helper.huawei_run_df(include_noise=False, include_refinements=False, 
    valid_x_columns=["fine_log_cluster_template", "coarse_log_cluster_template"])
huawei_accuracy_df2 = calculate_accuracies_per_percentiles(
    relevant_run_df=relevant_huawei_run_df2, 
    k=5, num_percentiles=10, num_input_percentiles=10,
    percentile_names=percentile_names,
    local_mlflow_dir=mlflow_helper.local_mlflow_dir)

In [ ]:
plot_accuracies_per_percentiles(
    show_plot=False,
    relevant_run_df=relevant_huawei_run_df2[relevant_huawei_run_df2["data_params_ModelConfigbase_hidden_embeddings_trainable"] == "False"], 
    accuracy_df=huawei_accuracy_df2[(huawei_accuracy_df2["type"] == "avg_input_frequencies_percentile") | (huawei_accuracy_df2["type"] == "unknown_inputs_percentile")],
    comparison_column="data_params_SequenceConfigx_sequence_column_name",
    comparison_column_order=["log_cluster_template", "coarse_log_cluster_template"],
    share_y=True)

In [ ]:
grouped_df = (
    pd.merge(
        relevant_huawei_run_df2[relevant_huawei_run_df2['data_tags_model_type'] != 'causal2'], 
        huawei_accuracy_df2[(huawei_accuracy_df2["type"] == "avg_input_frequencies_percentile") | (huawei_accuracy_df2["type"] == "unknown_inputs_percentile")],
        left_on="info_run_id", right_on="run_id")
    .groupby(
        [
            "data_tags_model_type",
            "data_params_ModelConfigbase_hidden_embeddings_trainable",
            "data_params_SequenceConfigx_sequence_column_name",
            "info_run_id",
            "type",
            "percentile",
        ],
        as_index=False,
    )
    .agg({"accuracy": max,})
)
g = sns.relplot(
    data=grouped_df,
    x="percentile",
    y="accuracy",
    row="type",
    col="data_tags_model_type",
    hue="data_params_SequenceConfigx_sequence_column_name",
    style="data_params_ModelConfigbase_hidden_embeddings_trainable",
    kind="line",
    palette=None,
)
g.set_titles("Type: {row_name}, Model: {col_name}").set_axis_labels(
    "", "accuracy"
)
for ax in g.axes.flatten():
    ax.tick_params(labelbottom=True)
plt.show()

In [ ]:
relevant_mimic_run_df.to_pickle("percentile_relevant_mimic_run_df.pkl")
relevant_huawei_run_df.to_pickle("percentile_relevant_huawei_run_df.pkl")
relevant_mimic_run_df2.to_pickle("percentile_relevant_mimic_run_df2.pkl")
relevant_huawei_run_df2.to_pickle("percentile_relevant_huawei_run_df2.pkl")

mimic_accuracy_df.to_pickle("percentile_mimic_accuracy_df.pkl")
mimic_accuracy_df2.to_pickle("percentile_mimic_accuracy_df2.pkl")
huawei_accuracy_df.to_pickle("percentile_huawei_accuracy_df.pkl")
huawei_accuracy_df2.to_pickle("percentile_huawei_accuracy_df2.pkl")

In [ ]:
relevant_mimic_run_df = pd.read_pickle("percentile_relevant_mimic_run_df.pkl")
relevant_huawei_run_df = pd.read_pickle("percentile_relevant_huawei_run_df.pkl")
relevant_mimic_run_df2 = pd.read_pickle("percentile_relevant_mimic_run_df2.pkl")
relevant_huawei_run_df2 = pd.read_pickle("percentile_relevant_huawei_run_df2.pkl")

mimic_accuracy_df = pd.read_pickle("percentile_mimic_accuracy_df.pkl")
mimic_accuracy_df2 = pd.read_pickle("percentile_mimic_accuracy_df2.pkl")
huawei_accuracy_df = pd.read_pickle("percentile_huawei_accuracy_df.pkl")
huawei_accuracy_df2 = pd.read_pickle("percentile_huawei_accuracy_df2.pkl")